In [2]:
import numpy as np

# Given simulation parameters
transmission_rates = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
K_values = range(50, 301, 50) # Number of devices, range from 50 to 300 in steps of 50
task_data_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbits
max_delay = 0.5  # in seconds
num_of_MECs = 10
MEC_compute_capacity = 4  # in GHz
transmission_power = 0.5  # in Watts
H_m = 1e-26  # energy consumption factor (example value)

# Constants for MEC server computations
B = 10**6  # Bandwidth in Hz (assuming 1 MHz)
D0 = 10**-9  # Noise power spectral density in W/Hz (assuming -90 dBm/Hz)
alpha = 4  # Path loss index (common urban value for cellular networks)
distance_um = 100  # Distance between user u and MEC server m in meters (example value)
f_m = 4 * 10**9  # Server CPU frequency in Hz (4 GHz)
R_u = 1000  # CPU cycles required to calculate 1-bit of task data (example value)
e_m = 10**-11  # Energy consumption needed to calculate 1-bit of task data for MEC server in J/bit (example value)

# Simulation function to calculate local computing
def simulate_local_computing(k, C_k, D_k, P_k):
    """
    Simulate local computing for 'k' users with given CPU power 'C_k', task data 'D_k', and offloading ratio 'P_k'.
    
    :param k: Number of devices (users)
    :param C_k: CPU power for devices
    :param D_k: Task data size for devices
    :param P_k: Offloading ratio for devices
    :return: Tuple of total time cost and energy consumption for local computing
    """
    # Convert task data size from Mbits to bits for computation
    D_k_bits = D_k * 1e6
    
    # Calculate local computing time cost and energy consumption
    TC_l = D_k_bits * (1 - P_k) / (C_k * 1e9)  # C_k converted from GHz to Hz for the calculation
    EC_l = H_m * (C_k**2) * D_k_bits * (1 - P_k)
    
    # Assuming equal weights for time cost and energy consumption
    q_kt = q_ke = 1
    WC_l = q_kt * TC_l + q_ke * EC_l
    
    return TC_l, EC_l, WC_l

In [3]:
# To store the results for each K value
simulation_results = {}

# Run the simulation for each K value
for k in K_values:
    C_k_values = np.random.choice(MEC_compute_capacity, k)  #CPU power for each device
    D_k_values = np.random.choice(task_data_sizes, k)  # Random task data size for each device
    P_k_values = np.random.rand(k)  # Random offloading ratio for each device
    
    # Run the simulation for current K value
    time_costs, energy_consumptions, weighted_costs = zip(*[simulate_local_computing(k, C_k, D_k, P_k)
                                                            for C_k, D_k, P_k in zip(C_k_values, D_k_values, P_k_values)])
    
    # Store the results
    simulation_results[k] = {
        'total_time_cost': sum(time_costs),
        'total_energy_consumption': sum(energy_consumptions),
        'total_weighted_cost': sum(weighted_costs)
    }

simulation_results

C:\Users\YANSILIYU\AppData\Local\Temp\ipykernel_4052\301260524.py:28: RuntimeWarning: divide by zero encountered in scalar divide
  TC_l = D_k_bits * (1 - P_k) / (C_k * 1e9)  # C_k converted from GHz to Hz for the calculation


{50: {'total_time_cost': inf,
  'total_energy_consumption': 2.1695869632486666e-17,
  'total_weighted_cost': inf},
 100: {'total_time_cost': inf,
  'total_energy_consumption': 5.526775979101831e-17,
  'total_weighted_cost': inf},
 150: {'total_time_cost': inf,
  'total_energy_consumption': 6.695901973556453e-17,
  'total_weighted_cost': inf},
 200: {'total_time_cost': inf,
  'total_energy_consumption': 8.627633704063043e-17,
  'total_weighted_cost': inf},
 250: {'total_time_cost': inf,
  'total_energy_consumption': 1.0352780453106327e-16,
  'total_weighted_cost': inf},
 300: {'total_time_cost': inf,
  'total_energy_consumption': 1.2166063685378085e-16,
  'total_weighted_cost': inf}}

In [6]:
import numpy as np

# Simulation parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
number_of_devices = range(50, 301, 50)
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbit
max_delay = 0.5  # in seconds
device_computing_capacity = [0.5, 1]  # in GHz
num_of_mecs = 10
mec_computing_capacity = 4  # in GHz
transmission_power = 0.5  # in Watts

# Constants for the energy consumption model
H_m = 0.01 # Consumption factor for the mobile device, needs to be defined
alpha_lt = 0.1  # Weighted coefficient of local time cost, needs to be defined
alpha_le = 0.002  # Weighted coefficient of local energy consumption, needs to be defined

# Constants for MEC server computations
B = 10**6  # Bandwidth in Hz (assuming 1 MHz)
D0 = 10**-9  # Noise power spectral density in W/Hz (assuming -90 dBm/Hz)
alpha = 4  # Path loss index (common urban value for cellular networks)
distance_um = 100  # Distance between user u and MEC server m in meters (example value)
f_m = 4 * 10**9  # Server CPU frequency in Hz (4 GHz)
R_u = 1000  # CPU cycles required to calculate 1-bit of task data (example value)
e_m = 10**-11  # Energy consumption needed to calculate 1-bit of task data for MEC server in J/bit (example value)

# Task offloading model
def offload_task(Dk, Pk, Ck):
    """
    Offloads the task based on the probability Pk.
    :param Dk: Data size of the task.
    :param Pk: Offloading probability for user k.
    :param Ck: Computing power of the device for user k.
    :return: Tuple of time cost and energy consumption.
    """
    # Time Cost of Local Execution
    TC_lk = Dk * (1 - Pk) / Ck
    # Energy Consumption of Local Execution
    EC_lk = H_m * Ck**2 * Dk * (1 - Pk)
    
    # Weighted Overhead
    WC_lk = alpha_lt * TC_lk + alpha_le * EC_lk
    
    return TC_lk, EC_lk, WC_lk

# Function to calculate channel gain between user u and MEC server m
def channel_gain(distance, alpha):
    return (distance)**(-alpha)

# Function to calculate transmission rate V_u (Equation 4)
def transmission_rate(TP_u, distance_um, D0, B, alpha):
    IG_um = channel_gain(distance_um, alpha)
    return B * np.log2(1 + TP_u * IG_um / (D0 * B))

# Function to calculate the uplink transmission time overhead TC_e1 (Equation 5)
def uplink_transmission_time(D_u, P_u, V_u):
    return (D_u * P_u) / V_u

# Function to calculate the task processing time cost at MEC server TC_e2 (Equation 6)
def mec_processing_time(D_u, P_u, R_u, f_m):
    return (D_u * P_u * R_u) / f_m

# Function to calculate the total time cost TC_total (Equation 7)
def total_time_cost(TC_e1, TC_e2):
    return TC_e1 + TC_e2

# Function to calculate the processing energy cost EC_e (Equation 8)
def processing_energy_cost(TP_u, D_u, P_u, V_u, e_m):
    return TP_u * (D_u * P_u / V_u) + D_u * P_u * e_m

# Run simulation for each device
for K in number_of_devices:
    for Ck in device_computing_capacity:
        for Dk in task_data:
            # Convert task data size from Mbit to bit for calculations
            Dk_bits = Dk * 10**6  
            # Simulate for different offloading probabilities
            for Pk in np.linspace(0, 1, 11):  # 11 different probabilities from 0 to 1
                # Compute local execution costs
                TC_lk, EC_lk, WC_lk = offload_task(Dk_bits, Pk, Ck * 10**9)  # Convert Ck from GHz to Hz
                
                # Compute MEC server execution costs if the task is offloaded
                if Pk > 0:  # Task is partially or fully offloaded
                    TP_u = transmission_power  # in Watts
                    V_u = transmission_rate(TP_u, distance_um, D0, B, alpha)
                    TC_e1 = uplink_transmission_time(Dk_bits, Pk, V_u)
                    TC_e2 = mec_processing_time(Dk_bits, Pk, R_u, f_m)
                    TC_total = total_time_cost(TC_e1, TC_e2)
                    EC_e = processing_energy_cost(TP_u, Dk_bits, Pk, V_u, e_m)
                else:  # No offloading
                    TC_total = TC_lk  # Only local execution time is considered
                    EC_e = EC_lk  # Only local execution energy is considered
                
                # Print results for this iteration
                print(f"K: {K}, Ck: {Ck} GHz, Dk: {Dk} Mbit, Pk: {Pk:.2f}, "
                      f"Local Time Cost: {TC_lk:.2f} s, Local Energy Consumption: {EC_lk:.2f} J, "
                      f"MEC Time Cost: {TC_total:.2f} s, MEC Energy Consumption: {EC_e:.2f} J, "
                      f"Weighted Overhead: {WC_lk:.2f}")

K: 50, Ck: 0.5 GHz, Dk: 5 Mbit, Pk: 0.00, Local Time Cost: 0.01 s, Local Energy Consumption: 12499999999999998951424.00 J, MEC Time Cost: 0.01 s, MEC Energy Consumption: 12499999999999998951424.00 J, Weighted Overhead: 25000000000000000000.00
K: 50, Ck: 0.5 GHz, Dk: 5 Mbit, Pk: 0.10, Local Time Cost: 0.01 s, Local Energy Consumption: 11249999999999999475712.00 J, MEC Time Cost: 69315.02 s, MEC Energy Consumption: 34657.45 J, Weighted Overhead: 22500000000000000000.00
K: 50, Ck: 0.5 GHz, Dk: 5 Mbit, Pk: 0.20, Local Time Cost: 0.01 s, Local Energy Consumption: 10000000000000000000000.00 J, MEC Time Cost: 138630.03 s, MEC Energy Consumption: 69314.89 J, Weighted Overhead: 20000000000000000000.00
K: 50, Ck: 0.5 GHz, Dk: 5 Mbit, Pk: 0.30, Local Time Cost: 0.01 s, Local Energy Consumption: 8749999999999998427136.00 J, MEC Time Cost: 207945.05 s, MEC Energy Consumption: 103972.34 J, Weighted Overhead: 17499999999999997952.00
K: 50, Ck: 0.5 GHz, Dk: 5 Mbit, Pk: 0.40, Local Time Cost: 0.01 s, L